# IMPORTS

In [ ]:
import os
import time
import getpass
import chromadb
import openai

from datetime import datetime
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

# Chroma vectorstore and Embedding model

Initialize existing persisting storage.

IMPORTANT: make sure to have loaded some documents to the vector database. This can be done by running the ``load_data.py`` in /src.

In [ ]:
PATH_DB = 'db'
COLLECTION_NAME = 'policy'

# Instantiate a persistent chroma client in the persist_directory.
# This will automatically load any previously saved collections.
client_db = chromadb.PersistentClient(path=PATH_DB)

# Get the collection.
collection = client_db.get_collection(
    name=COLLECTION_NAME, 
    embedding_function=OllamaEmbeddingFunction(
        model_name="mxbai-embed-large",
        url="http://localhost:11434/api/embeddings",
    )
)
collection.get()["metadatas"]

# Retrieval and generation: Create an agent2agent dialogue pipeline

In [ ]:
def build_prompt(message, docs, is_angel=False):
    """
    Define system prompts for the two agents
    """

    user_prompt = f"""Heres the some documents and a list of the previous debates:
        Documents: {docs}
        Previous debates: {message}
    """

    restrictions = f"""You must build your arguments only on the information in the documents.
        Answer with as much information as you can find. Keep in mind that some documents may be old and no longer valid.
        If a document mentions that it replaces previous documents via its file number, take into account which document is the current valid one and which should prevail.
        If you lack information, the information is ambiguous, or the answer for any other reason is uncertain or unclear, state that “the answer is not clear” and explain why.
        For any answer you give, you are always forced to give supporting quotes and refer to the source documents.
        Answer in Swedish.
    """

    output_format = f"""
        **FIELD NAME (e.g. finance, recruitment)**

            ***document title [reference number/diarienummer]***
                - Quotation and page number. Include your interpretation of the quotation.

        
        ***Sammanfattning***
        
        Summary of the arguments using the quotes. Break it up in nice and readable paragraphs.\n\n
    """
    
    if not is_angel:
        devil_system_prompt = [
            {
                "role": "system",
                "content": f"""You are the Devil's Advocate. You will have a debate with the Angel's Advocate.
                    You'll be provided with a list of documents and all the previous debates.
                    The documents consists of a number of governing documents from a university.
                    Your task is to exploit loopholes in the university´s governing documents for the user's benefit. 
                    Be specific and detailed on what and how you can exploit the loopholes.
                    
                    {restrictions}

                    #RESPONSEFOMAT
                    Provide your response in the following format: 
                    # DEVIL:
                    {output_format}
                    """
            }, 
            {"role": "user", "content": user_prompt}
        ]
        return devil_system_prompt
    else:
        angel_system_prompt = [
            {
                "role": "system",
                "content": f"""You are the Angel's Advocate. You will have a debate with the Devil's Advocate.
                    You'll be provided with a list of documents and all the previous debates.
                    The documents consists of a number of governing documents from a university.
                    Your task is to argue against the Devil's Advocates arguments to prevent the exploitation of loopholes in the university´s governing documents.

                    {restrictions}

                    #RESPONSEFOMAT
                    Provide your response in the following format:
                    # ANGEL:
                    {output_format}
                """
            }, 
            {"role": "user", "content": user_prompt}
        ]
    return angel_system_prompt

def stream_response(response: ChatCompletion) -> str:
    """Stream response to output, and convert the response to string.

    Args:
        response (ChatCompletion): Response from OpenAI chatbot.

    Returns:
        str: Response message in string.
    """

    message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            message += (chunk.choices[0].delta.content)
            print(chunk.choices[0].delta.content, end="")
    print("\n\n")
    return message

def get_response(agent: OpenAI, model: str, temperature: float, **kwargs) -> str:
    """Creates a model response for the given chat conversation and streams the response.

    Args:
        agent (OpenAI): An openai client instance.
        model (str): model name e.g. gpt-4o-mini
        temperature (float, optional): What sampling temperature to use, between 0 and 2. 
            Higher values like 0.8 will make the output more random, while lower values like 0.2 
            will make it more focused and deterministic. Defaults to 0.76.

    Returns:
        str: Response message in string.
    """

    response = agent.chat.completions.create(
        model=model,
        messages=build_prompt(**kwargs),
        stream=True,
        temperature=temperature
    )
    message = stream_response(response)
    return message

def run_debate(
    task_prompt: str, 
    model: str, 
    temperature: float = 0.76, 
    num_rounds: int = 3
) -> str:
    
    # Set OpenAI API key
    openai.api_key = os.getenv('OPENAI_API_KEY') if 'OPENAI_API_KEY' in os.environ else getpass.getpass("Enter your OpenAI API key: ")

    # Instantiate the openai clients for each agent. 
    devil_agent = OpenAI()
    angel_agent = OpenAI()
    boss_agent = OpenAI()
    
    # Query the collection to get the 5 most relevant results
    task_prompt +=  "\nList the reference number/diarienummer for the documents."
    docs = collection.query(query_texts=[task_prompt], include=["documents", "metadatas"])
    print(f"=> Nr. of relevant documents: {len(docs)}")

    if not docs:
        raise ValueError(f"No relevant docs were retrieved!")
    
    # Prepare dialogue.
    message = "# QUERY:\n" + task_prompt
    dialogue = [message]
    print("\n" + message)

    for round_num in range(num_rounds):
        print(f"__________Turn number {round_num+1}__________\n")
        
        ## Devil agent's turn.
        message = get_response(devil_agent, model, temperature, message=dialogue, docs=docs, is_angel=False)
        dialogue.append(message)
        
        ## Angel agent's turn.
        message = get_response(angel_agent, model, temperature, message=dialogue, docs=docs, is_angel=True)
        dialogue.append(message)

        ## Boss agent's turn.
        boss_response = boss_agent.chat.completions.create(
            model=model,
            messages=[{
                "role": "user", 
                "content": f"""
                    Analyze the following debate arguments {dialogue} and make a final decision.
                    Start with "\n # BOSS:\n" followed by your response.
                    Answer in Swedish.
                """
            }],
            stream=True
        )
        boss_message = stream_response(boss_response)
        dialogue.append(boss_message)

    return "\n\n".join(dialogue)

## Query it

In [ ]:
## Define initial task prompt for the devil agent
task_alcohol = """Discuss why the governing documents does allow you to drink beer during working hours."""

task_prof = """Jag är anställd på Chalmers och vill rekrytera en framgångsrik professor från USA till Chalmers.
Vilka dokument är relevanta för mig att ha i åtanke när jag bjuder in professorn för att sälja in Chalmers och en ledig tjänst på universitetet?
Jag vill:
    1. flyga över forskaren från USA,
    2. bjuda in honom och hans fru på middag,
    3. låta honom bo på det finaste hotellet i Göteborg.
Helst vill jag att Chalmers betalar för alltihop, eftersom just den här professorn och vad han kan tillföra skulle vara ovärderligt för Chalmers. 
"""

MODEL = "gpt-4o-mini"
# MODEL = "gpt-3.5-turbo"

## Run the debate
start_time = time.time() # Start the timer

debate_dialogue = run_debate(task_prof, MODEL, num_rounds=1)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

## Save the dialogue to a .txt file
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"out/debate_dialogue_{current_time}.md"
with open(file_name, "w") as file:
    file.write(debate_dialogue)
    file.write(f"\n\n ***Time taken: {elapsed_time:.2f} seconds***")

print(f"=> Debate dialogue saved to {file_name}")
print(f"=> Time taken: {elapsed_time:.2f} seconds")